In [18]:
import torch
from src import Mutator, get_model, get_dataloaders, perplexity

In [33]:
args = {
    "from_tokenizer": "Rostlab/prot_bert_bfd",
    "from_model": "Rostlab/prot_bert_bfd",
    "from_adapters": "./models/hps/LR0.001_BS256_P0.05/best/",
    "set": "./data/non_thermo.csv",
    "min_length": None,
    "max_length": None,
    "global_batch_size": 512,
    "local_batch_size": 128,
    "num_workers": 8,
    "mask": False,
    "p": 0,
    "n_epochs": 10
}

model = get_model(args)
dataloader, tokenizer = get_dataloaders(args, get_training_and_validation=False, get_tokenizer=True)
Mutator.configure(model, tokenizer, 3, 3, 0.1)
mutator = Mutator()

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
sequence = "MVMLMWAYLSLLVRKDSQHDCRRTKEGNSREETSVGVCRDSGTDTTGRRGEGSARGKEKEAQPLFQCR"
tokens = tokenizer([' '.join(list(sequence))], return_tensors="pt")
input_ids, attention_mask = tokens.input_ids, tokens.attention_mask

for i in range(5):
    print(perplexity(model, input_ids, attention_mask=attention_mask))
    out = mutator(input_ids=input_ids, attention_mask=attention_mask)
    for el in tokenizer.batch_decode(out.mutated_ids, skip_special_tokens=True):
        print(el.replace(" ", ""))
    input_ids = out.mutated_ids

tensor([1.9203])
MVMLMWAYLSLLVRKDSQHDCRRTKEGNSREETSRGSCRDSGTDTTGRRGEGSARGKEKEAQPLFQCR
tensor([1.8621])
MKMLMWAYLSLLVRKDSQHDSRRTKEGNSREETSRGSCRDSGTDTTGRRGEGSARGKEKEAQPLFQCR
tensor([1.8599])
MKMLMWAYLSLLVRKDSQHDSRRTKEGNSREETSRGSCRDSGTGTTGRRGEGSARGKEKEAQPLFQCR
tensor([1.8417])
MKMLMWAYLSLLVRKDSSHDSRRTKEGNSREETSRGSCRDSGTGTTGRRGEGSARGKEKEAQPLFQRR
tensor([1.9013])
MKMLMWAYLSLLVRRDSSHDSRRTKEGNSREETSRGSCRGSGTGTTGRRGEGSARGKEKEAQPLFQRR
